# Tool Calling

Tools are functions that can be invoked by an agent to perform tasks.

They are organized into tool groups and registered with specific providers. Each tool group represents a collection of related tools from a single provider. They are organized into groups so that state can be externalized, so that the collection operates on the same state. An example of this would be a “db_access” tool group that contains tools for interacting with a database. “list_tables”, “query_table”, “insert_row” could be examples of tools in this group.

Tools are treated as any other resource in llama stack like models. You can register them, have providers for them etc.

When instantiating an agent, you can provide it a list of tool groups that it has access to.
Agent gets the corresponding tool definitions for the specified tool groups and passes them along to the model.

## Setup

In [1]:
# Imports
import os
from llama_stack_client import LlamaStackClient

# Select model
model = "sambanova/Meta-Llama-3.3-70B-Instruct"

In [2]:
# Create HTTP client
client = LlamaStackClient(base_url=f"http://localhost:{os.environ['LLAMA_STACK_PORT']}")

## Types of Tool Group providers
There are three types of providers for tool groups that are supported by Llama Stack.

1. Built-in providers.
2. Model Context Protocol (MCP) providers.
3. Client provided tools.

## Built-in providers
Built-in providers come packaged with Llama Stack. These providers provide common functionalities like web search, code interpretation, and computational capabilities.

### Web Search providers
There are three web search providers that are supported by Llama Stack.

- Brave Search.
- Bing Search.
- Tavily Search.

### Code Interpreter
The Code Interpreter allows execution of Python code within a controlled environment.

Features:

- Secure execution environment using bwrap sandboxing.
- Matplotlib support for generating plots.
- Disabled dangerous system operations.
- Configurable execution timeouts.

### WolframAlpha
The WolframAlpha tool provides access to computational knowledge through the WolframAlpha API.



## List available tool groups on the provider

In [3]:
from rich.pretty import pprint
for toolgroup in client.toolgroups.list():
    pprint(toolgroup)

ToolGroup(
│   identifier='builtin::code_interpreter',
│   provider_id='code-interpreter',
│   provider_resource_id='builtin::code_interpreter',
│   type='tool_group',
│   args=None,
│   mcp_endpoint=None
)

ToolGroup(
│   identifier='builtin::rag',
│   provider_id='rag-runtime',
│   provider_resource_id='builtin::rag',
│   type='tool_group',
│   args=None,
│   mcp_endpoint=None
)

ToolGroup(
│   identifier='builtin::websearch',
│   provider_id='tavily-search',
│   provider_resource_id='builtin::websearch',
│   type='tool_group',
│   args=None,
│   mcp_endpoint=None
)

ToolGroup(
│   identifier='builtin::wolfram_alpha',
│   provider_id='wolfram-alpha',
│   provider_resource_id='builtin::wolfram_alpha',
│   type='tool_group',
│   args=None,
│   mcp_endpoint=None
)

## Tool invocation
Tools can be invoked using the `invoke_tool` method.

```python
result = client.tool_runtime.invoke_tool(
    tool_name="<tool_name>", kwargs={"query": "What is the capital of France?"}
)
```

## Custom Tools

When you want to use tools other than the built-in tools, you just need to implement a python function with a docstring. The content of the docstring will be used to describe the tool and the parameters and passed along to the generative model.

NOTE: You should employ python docstrings to describe the tool and the parameters. It is important to document the tool and the parameters so that the model can use the tool correctly. It is recommended to experiment with different docstrings to see how they affect the model’s behavior.

In [4]:
def check_prime(n: int) -> bool:
    """
    Determines if a given integer is a prime number.

    :param n: The integer to check.
    :return: True if 'n' is prime, otherwise False.
    """
    if n < 2:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

## Agent with tools
Once defined, simply pass the tool to the agent config. Agent will take care of the rest (calling the model with the tool definition, executing the tool, and returning the result to the model for the next iteration).

In [5]:
from llama_stack_client.lib.agents.agent import Agent

# Example agent config with client provided tools
agent = Agent(
  client, 
  model=model,
  instructions="You are a helpful assistant",
  tools = [check_prime],
)

In [6]:
from llama_stack_client.lib.agents.event_logger import EventLogger

session_id = agent.create_session("custom-tool-session")

response = agent.create_turn(
    messages=[{"role": "user", "content": 'Is 25 a prime number?'}],
    session_id=session_id,
)
for log in EventLogger().log(response):
    log.print()

inference> {"n":25})
tool_execution> Tool:check_prime Args:{'n': 25.0}
tool_execution> Tool:check_prime Response:false
inference> This is because 25 can be divided by 5, so it's not a prime number.
